In [24]:
import numpy as np
import tqdm
policy_txt = '/Users/wangrui/Projects/Deep-Learning-Project_RL/Complex-YOLOv4/dataset/kitti/resnet50_policies_1.txt'
hr_txt = '/Users/wangrui/Projects/Deep-Learning-Project_RL/Complex-YOLOv4/dataset/kitti/fine_detector_boxid.txt'
lr_txt = '/Users/wangrui/Projects/Deep-Learning-Project_RL/Complex-YOLOv4/dataset/kitti/coarse_detector_boxid.txt'
id_txt ='/Users/wangrui/Projects/Deep-Learning-Project_RL/Complex-YOLOv4/dataset/kitti/val_id_16.txt'
gt_txt = '/Users/wangrui/Projects/Deep-Learning-Project_RL/Complex-YOLOv4/dataset/kitti/ground_truth_16.txt'
policy_mat = np.loadtxt(policy_txt)
hr_mat = np.loadtxt(hr_txt)
lr_mat = np.loadtxt(lr_txt)

id_mat = np.loadtxt(id_txt)
gt_mat = np.loadtxt(gt_txt)



In [25]:
def ap_per_class(tp, conf, pred_cls, target_cls):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")


def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [26]:
# def find_idx(this_po)
###### Ensemble RL
eval_len,_ = np.shape(policy_mat)
true_positives, pred_scores, pred_labels, labels = [],[],[],[]
gt_labels = []
for i in range(eval_len):
    this_po = policy_mat[i,:]
    this_index = np.where(np.logical_and(id_mat[:,0] == this_po[0], id_mat[:,1] == this_po[1]))
    this_index = this_index[0]
    if this_index.size == 0:
        continue
    gt_index = np.where(gt_mat[:,0]==this_index)[0]
    if len(gt_index)==1:
        gt_labels.append(gt_mat[gt_index,2])
    else:
        gt_labels+= list(gt_mat[gt_index,2])
        
    
    if this_po[2]>0:
        hr_index = np.where(hr_mat[:,0]==this_index)[0]
        
        if len(hr_index) == 1:
            true_positives.append(np.double(hr_mat[hr_index,2]>=0.5))
        else:
            true_positives += list(np.double(hr_mat[hr_index,2]>=0.5))
        gt_box_id = hr_mat[hr_index,5]
        labels += list(gt_mat[gt_index[gt_box_id.astype('int')],2])
        pred_scores += list(hr_mat[hr_index,3])
        pred_labels += list(hr_mat[hr_index,4])

    else:
        lr_index = np.where(lr_mat[:,0]==this_index)[0]
        if len(lr_index) == 1:
            true_positives.append(np.double(lr_mat[lr_index,2]>=0.5))
        else:
            true_positives += list(np.double(lr_mat[lr_index,2]>=0.5))
        gt_box_id = lr_mat[lr_index,5]
        labels += list(gt_mat[gt_index[gt_box_id.astype('int')],2])
        pred_scores += list(lr_mat[lr_index,3])
        pred_labels += list(lr_mat[lr_index,4])
        

#         sample_metrics += 
        
precision, recall, AP, f1, ap_class = ap_per_class(np.array(true_positives),np.array(pred_scores), np.array(pred_labels), np.array(gt_labels))
print('precision',precision)

/Users/wangrui/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Computing AP: 100%|██████████| 3/3 [00:00<00:00, 529.12it/s]

precision [0.8301105  0.47881356 0.55      ]


In [4]:
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

	>>>	 Class  (0): precision = 0.8209, recall = 0.8945, AP = 0.8529, f1: 0.8561
	>>>	 Class  (1): precision = 0.5409, recall = 0.7000, AP = 0.4607, f1: 0.6103
	>>>	 Class  (2): precision = 0.6528, recall = 0.7705, AP = 0.6253, f1: 0.7068


In [9]:
# def find_idx(this_po)
###### Ensemble RL
eval_len,_ = np.shape(policy_mat)
true_positives, pred_scores, pred_labels, labels = [],[],[],[]
gt_labels = []
for i in range(eval_len):
    this_po = policy_mat[i,:]
    this_index = np.where(np.logical_and(id_mat[:,0] == this_po[0], id_mat[:,1] == this_po[1]))
    this_index = this_index[0]
    if this_index.size == 0:
        continue
    gt_index = np.where(gt_mat[:,0]==this_index)[0]
    if len(gt_index)==1:
        gt_labels.append(gt_mat[gt_index,2])
    else:
        gt_labels+= list(gt_mat[gt_index,2])
        
    
    if np.random.rand()<0.2601:
        hr_index = np.where(hr_mat[:,0]==this_index)[0]
        
        if len(hr_index) == 1:
            true_positives.append(np.double(hr_mat[hr_index,2]>0.5))
        else:
            true_positives += list(np.double(hr_mat[hr_index,2]>0.5))
        gt_box_id = hr_mat[hr_index,5]
        labels += list(gt_mat[gt_index[gt_box_id.astype('int')],2])
        pred_scores += list(hr_mat[hr_index,3])
        pred_labels += list(hr_mat[hr_index,4])

    else:
        lr_index = np.where(lr_mat[:,0]==this_index)[0]
        if len(lr_index) == 1:
            true_positives.append(np.double(lr_mat[lr_index,2]>0.5))
        else:
            true_positives += list(np.double(lr_mat[lr_index,2]>0.5))
        gt_box_id = lr_mat[lr_index,5]
        labels += list(gt_mat[gt_index[gt_box_id.astype('int')],2])
        pred_scores += list(lr_mat[lr_index,3])
        pred_labels += list(lr_mat[lr_index,4])
        

#         sample_metrics += 
        
precision, recall, AP, f1, ap_class = ap_per_class(np.array(true_positives),np.array(pred_scores), np.array(pred_labels), np.array(gt_labels))
print('precision',precision)

/Users/wangrui/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Computing AP: 100%|██████████| 3/3 [00:00<00:00, 498.75it/s]

precision [0.78160159 0.29803922 0.3047619 ]


In [14]:
print('Ensemble')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

Ensemble
	>>>	 Class  (0): precision = 0.8392, recall = 0.9058, AP = 0.8719, f1: 0.8712
	>>>	 Class  (1): precision = 0.5952, recall = 0.7353, AP = 0.5301, f1: 0.6579
	>>>	 Class  (2): precision = 0.7353, recall = 0.8197, AP = 0.7194, f1: 0.7752


In [10]:
print('ALL random')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

ALL random
	>>>	 Class  (0): precision = 0.7816, recall = 0.8833, AP = 0.8065, f1: 0.8294
	>>>	 Class  (1): precision = 0.2980, recall = 0.4471, AP = 0.1812, f1: 0.3576
	>>>	 Class  (2): precision = 0.3048, recall = 0.5246, AP = 0.2085, f1: 0.3855


In [21]:
print('Reg1.6G')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

Reg1.6G
	>>>	 Class  (0): precision = 0.8139, recall = 0.8863, AP = 0.8332, f1: 0.8485
	>>>	 Class  (1): precision = 0.5434, recall = 0.7000, AP = 0.4589, f1: 0.6118
	>>>	 Class  (2): precision = 0.6133, recall = 0.7541, AP = 0.5868, f1: 0.6765


In [27]:
print('Resnet')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

Resnet
	>>>	 Class  (0): precision = 0.8301, recall = 0.8990, AP = 0.8604, f1: 0.8632
	>>>	 Class  (1): precision = 0.4788, recall = 0.6647, AP = 0.3946, f1: 0.5567
	>>>	 Class  (2): precision = 0.5500, recall = 0.7213, AP = 0.5536, f1: 0.6241


In [8]:
print('ALL low')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

ALL low
	>>>	 Class  (0): precision = 0.7567, recall = 0.8654, AP = 0.7713, f1: 0.8074
	>>>	 Class  (1): precision = 0.1929, recall = 0.3176, AP = 0.0957, f1: 0.2400
	>>>	 Class  (2): precision = 0.2692, recall = 0.4590, AP = 0.1702, f1: 0.3394


In [6]:
print('ALL high')
for idx, cls in enumerate(ap_class):
    print("\t>>>\t Class  ({}): precision = {:.4f}, recall = {:.4f}, AP = {:.4f}, f1: {:.4f}".format(cls, precision[idx], recall[idx], AP[idx], f1[idx]))

ALL high
	>>>	 Class  (0): precision = 0.8650, recall = 0.9342, AP = 0.9190, f1: 0.8982
	>>>	 Class  (1): precision = 0.6567, recall = 0.7765, AP = 0.6292, f1: 0.7116
	>>>	 Class  (2): precision = 0.8358, recall = 0.9180, AP = 0.8872, f1: 0.8750
